# Database Preparation

## Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Creating new data frame and csv from classy

 Creating new DataFrame and CSV from the original data from `classy_data.csv`, with columns:

| Spectra from 0.45 to 2.45 | Albedo | Names | Counts | Class_BDM (DeMeo) | Class_Asteroid_SF (Mahlke) |
|--------------------------|--------|-------|--------|-------------------|--------------------------|

In [ ]:
# Load the original CSV file
csv_file_path = 'classy_data.csv'
df = pd.read_csv(csv_file_path)

# Define the columns to keep in the specified order
selected_columns = list(df.columns[3:56]) + [df.columns[62], df.columns[2], df.columns[57], df.columns[114], df.columns[153]]

# Create a new DataFrame with the selected columns
df_selected = df[selected_columns]

# Save the new DataFrame to a CSV file
df_selected.to_csv('01-Base.csv', index=False)

# Print confirmation message
print("File '01-Base.csv' has been created successfully!")

In [ ]:
csv_file_path = '01-Base.csv'
df = pd.read_csv(csv_file_path)
df

In [ ]:
for i, col in enumerate(df.columns):
    print(f"Column {i}: {col}")

## Data exploration

Calculating completeness percentage and histrogram of the spectra and albedo. A sample removal was performed as well as a new CSV `02-Base.csv` excluding samples within classes O and X for further analysis, as class O contained only 3 samples and class X do not have any albedos.

In [ ]:
# Load the CSV file
csv_file_path = '01-Base.csv'
df = pd.read_csv(csv_file_path)

# Select the first 53 columns (0 to 52)
df_spectra = df.iloc[:, 0:53]

# Calculate overall missing data percentage
total_values = df_spectra.size
missing_values = df_spectra.isna().sum().sum()
missing_percentage = (missing_values / total_values) * 100
completeness_percentage = 100 - missing_percentage

# Convert first_valid_index() and last_valid_index() to integer positions
first_valid = df_spectra.apply(lambda row: row.first_valid_index(), axis=1)
last_valid = df_spectra.apply(lambda row: row.last_valid_index(), axis=1)

# Convert column labels to integer indices
column_index_map = {col: idx for idx, col in enumerate(df_spectra.columns)}
first_valid_idx = first_valid.map(column_index_map)
last_valid_idx = last_valid.map(column_index_map)

# Initialize counters for left and right missing values
left_missing, right_missing = 0, 0
incomplete_samples = df_spectra[df_spectra.isna().any(axis=1)]  # Only consider incomplete samples

for i, row in incomplete_samples.iterrows():
    first_non_nan = first_valid_idx[i]
    last_non_nan = last_valid_idx[i]

    if pd.notna(first_non_nan) and pd.notna(last_non_nan):  # Ensure valid indices
        left_missing += row.iloc[:int(first_non_nan)].isna().sum()   # Count NaNs before the first valid
        right_missing += row.iloc[int(last_non_nan) + 1:].isna().sum()  # Count NaNs after the last valid

# Compute percentages for left and right missing values
total_incomplete_missing = left_missing + right_missing
left_percentage = (left_missing / total_incomplete_missing) * 100 if total_incomplete_missing > 0 else 0
right_percentage = (right_missing / total_incomplete_missing) * 100 if total_incomplete_missing > 0 else 0

# Print results
print(f"Overall missing data percentage: {missing_percentage:.2f}%")
print(f"Overall completeness percentage: {completeness_percentage:.2f}%")
print(f"Missing data on the left side (excluding fully complete samples): {left_percentage:.2f}%")
print(f"Missing data on the right side (excluding fully complete samples): {right_percentage:.2f}%")


In [ ]:
# Calculate the percentage of completed (non-missing) values per column
completion_percentage = df_spectra.notna().sum() / len(df_spectra) * 100
total_samples_per_column = df_spectra.notna().sum()  # Total non-missing samples per column

# Plot histogram (bar chart) of completion percentage per column
fig, ax = plt.subplots(figsize=(13.5, 3.8))  # Slightly increase width, keep height

# Make bars slightly wider
bar_width = 0.89  # Slightly increase width (default is ~0.8)
bars = ax.bar(df_spectra.columns.astype(str), completion_percentage, color="darkcyan", align='center', width=bar_width)

# Add total samples as labels inside the bars with DejaVu Serif font
for bar, count in zip(bars, total_samples_per_column):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, height - 5, str(count),
            ha='center', va='top', color='white', rotation=90, fontsize=11.7, fontname='DejaVu Serif')  # Apply font

# Customize the plot with DejaVu Serif for axis labels and numbers
ax.set_xlabel("Spectral Coverage (μm)", fontsize=14, fontname='DejaVu Serif')
ax.set_ylabel("Completion Percentage (%)", fontsize=14, fontname='DejaVu Serif')

# Adjust x-tick labels to be centered under the corresponding bars
ax.set_xticks(range(len(df_spectra.columns)))  # Ensure correct x-tick positions
ax.set_xticklabels(df_spectra.columns.astype(str), rotation=90, fontsize=13, fontname='DejaVu Serif', ha='center')

ax.grid(axis="y", linestyle="--", alpha=0.7)

ax.set_xlim(left=-0.7, right=len(df_spectra.columns) - 0.4)
# Remove title
ax.set_title("")

# Save the figure as PDF
pdf_path = "completion_histogram.pdf"
plt.savefig(pdf_path, format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
# Calculate completeness of column 53 (Albedo)
albedo_column = df.iloc[:, 53]
num_samples_with_data = albedo_column.notna().sum()
num_samples_total = len(albedo_column)
completeness_percentage = num_samples_with_data / num_samples_total * 100
print(f"Number of samples with Albedo data: {num_samples_with_data}")
print(f"Completeness of column 53 (Albedo): {completeness_percentage:.2f}%")

In [ ]:
# Load the CSV file
csv_file_path = '01-Base.csv'
df = pd.read_csv(csv_file_path)

# Extract albedo classification column
albedo_column = df.iloc[:, 53]

# Identify samples to exclude based on Mahlke taxonomy
excluded_samples = df[df['class_asteroid_sf'].isin(['O', 'X'])]
excluded_indices = excluded_samples.index

# Mahlke Scheme
original_mahlke_classes = df['class_asteroid_sf'].copy()
class_mapping_mahlke = {label: idx for idx, label in enumerate(np.unique(df['class_asteroid_sf']))}
class_counts_mahlke = df['class_asteroid_sf'].value_counts()

# Count occurrences of albedo values for each classification scheme
albedo_counts_mahlke = df.groupby('class_asteroid_sf')[albedo_column.name].count()

# Select the first 53 columns (0 to 52) for spectral data
df_spectra = df.iloc[:, 0:53]

# Calculate completeness percentage for each sample (row)
sample_completeness = df_spectra.notna().sum(axis=1) / df_spectra.shape[1] * 100

# Categorize samples into completeness ranges (100%, 90%, 80%, etc.)
bins = list(range(0, 110, 10))  # Ranges: 0-10%, 10-20%, ..., 90-100%
labels = [f"{i}%" for i in range(0, 100, 10)]

# Assign each sample to a category
df_spectra["Completeness Category"] = pd.cut(sample_completeness, bins=bins, labels=labels, right=True)

# Count samples per category and sort from most to least
completeness_counts = df_spectra["Completeness Category"].value_counts().sort_index(ascending=False)

# Convert to DataFrame for display
completeness_table = pd.DataFrame({"Completeness Level": completeness_counts.index, "Number of Samples": completeness_counts.values})

# Print side by side with tabs
print("Mahlke Scheme\tSpectra\tAlbedo")
print("-------------------------------------")

# Convert mappings to sorted lists for structured output (keep original class labels)
mahlke_items = sorted(class_mapping_mahlke.items(), key=lambda x: x[1])

# Print classification in a tabular format with albedo counts
for class_label_mahlke, index_mahlke in mahlke_items:
    count_mahlke = class_counts_mahlke.get(class_label_mahlke, 0) if class_label_mahlke else ""
    albedo_mahlke = albedo_counts_mahlke.get(class_label_mahlke, 0) if class_label_mahlke else ""

    print(f"{class_label_mahlke} -> {index_mahlke}  \t{count_mahlke}\t{albedo_mahlke}")

# Print completeness table
print("\nCompleteness Table:")
print(completeness_table)


In [ ]:
# Load the CSV file
csv_file_path = '01-Base.csv'
df = pd.read_csv(csv_file_path)

# Extract albedo classification column
albedo_column = df.iloc[:, 53]

# Identify samples to exclude based on Mahlke taxonomy
excluded_samples = df[df['class_asteroid_sf'].isin(['O', 'X'])]
excluded_indices = excluded_samples.index

# Mahlke Scheme (excluding O and X classes)
df_filtered = df.drop(index=excluded_indices)
original_mahlke_classes = df_filtered['class_asteroid_sf'].copy()
class_mapping_mahlke = {label: idx for idx, label in enumerate(np.unique(df_filtered['class_asteroid_sf']))}
class_counts_mahlke = df_filtered['class_asteroid_sf'].value_counts()

# Sort by class_asteroid_sf and then by name (column 54)
df_filtered = df_filtered.sort_values(by=['class_asteroid_sf', df_filtered.columns[54]])

# Save filtered and sorted DataFrame to new CSV
filtered_csv_path = '02-Base.csv'
df_filtered.to_csv(filtered_csv_path, index=False)

# Count occurrences of albedo values for each classification scheme
albedo_counts_mahlke = df_filtered.groupby('class_asteroid_sf')[albedo_column.name].count()

# Select the first 53 columns (0 to 52) for spectral data
df_spectra = df_filtered.iloc[:, 0:53]

# Calculate completeness percentage for each sample (row)
sample_completeness = df_spectra.notna().sum(axis=1) / df_spectra.shape[1] * 100

# Categorize samples into completeness ranges (100%, 90%, 80%, etc.)
bins = list(range(0, 110, 10))  # Ranges: 0-10%, 10-20%, ..., 90-100%
labels = [f"{i}%" for i in range(0, 100, 10)]

# Assign each sample to a category
df_spectra["Completeness Category"] = pd.cut(sample_completeness, bins=bins, labels=labels, right=True)

# Count samples per category and sort from most to least
completeness_counts = df_spectra["Completeness Category"].value_counts().sort_index(ascending=False)

# Convert to DataFrame for display
completeness_table = pd.DataFrame({"Completeness Level": completeness_counts.index, "Number of Samples": completeness_counts.values})

# Print side by side with tabs
print("Mahlke Scheme\tSpectra\tAlbedo")
print("-------------------------------------")

# Convert mappings to sorted lists for structured output (keep original class labels)
mahlke_items = sorted(class_mapping_mahlke.items(), key=lambda x: x[1])

# Print classification in a tabular format with albedo counts
for class_label_mahlke, index_mahlke in mahlke_items:
    count_mahlke = class_counts_mahlke.get(class_label_mahlke, 0) if class_label_mahlke else ""
    albedo_mahlke = albedo_counts_mahlke.get(class_label_mahlke, 0) if class_label_mahlke else ""

    print(f"{class_label_mahlke} -> {index_mahlke}  \t{count_mahlke}\t{albedo_mahlke}")

# Print completeness table
print("\nCompleteness Table:")
print(completeness_table)

Ok, we are now ready to next stage with the new CSV `02-Base.csv`

In [ ]:
# Load the CSV file
df = pd.read_csv('02-Base.csv')

# Select the first 53 columns (0 to 52)
df_spectra = df.iloc[:, 0:53]

# Extract the class column
class_column = df.iloc[:, 57]

# Check for complete spectra (no missing values in columns 0 to 52)
complete_spectra_mask = df_spectra.iloc[:, 0:53].notna().all(axis=1)

# Count the number of complete spectra per class
class_counts = class_column[complete_spectra_mask].value_counts().reset_index()
class_counts.columns = ['Class', 'Complete Spectra Count']

# Sum all counts
total_count = class_counts['Complete Spectra Count'].sum()

# Display the result
print(class_counts)
print("\nTotal number of complete spectra samples:", total_count)

In [ ]:
# Load the CSV file
df = pd.read_csv('02-Base.csv')
# Calculate completeness of column 53 (Albedo)
albedo_column = df.iloc[:, 53]
num_samples_with_data = albedo_column.notna().sum()
num_samples_total = len(albedo_column)
completeness_percentage = num_samples_with_data / num_samples_total * 100
print(f"Number of samples with Albedo data: {num_samples_with_data}")
print(f"Completeness of column 53 (Albedo): {completeness_percentage:.2f}%")